In [1]:
from pymongo import MongoClient

# Connect to your local MongoDB instance
client = MongoClient("mongodb://localhost:27017/")

# Create/use a database
db = client["testDB"]

# Create/use a collection
collection = db["testCollection"]

# Insert a document
collection.insert_one({"name": "Federico", "language": "Python", "likes": 42})

# Find a document
result = collection.find_one({"name": "Federico"})
print("Found document:", result)

# Delete the document
#collection.delete_one({"name": "Federico"})

Found document: {'_id': ObjectId('67fed3530038bcfda9a09eff'), 'name': 'Federico', 'language': 'Python', 'likes': 42}


In [3]:
from pymongo import MongoClient
import numpy as np

client = MongoClient("mongodb://localhost:27017/")
db = client["vector_db"]
collection = db["embeddings"]

# Drop collection to start fresh (optional)
collection.drop()

# Insert some dummy vector entries
documents = [
    {"name": "cat", "embedding": np.random.rand(5).tolist()},
    {"name": "dog", "embedding": np.random.rand(5).tolist()},
    {"name": "airplane", "embedding": np.random.rand(5).tolist()},
]
collection.insert_many(documents)

# Create a vector index (new in MongoDB 7.0)
db.command({
    "createIndexes": "embeddings",
    "indexes": [{
        "name": "embedding_vector_index",
        "key": {"embedding": "cosine"},
        "type": "vectorSearch",
        "numDimensions": 5,
        "similarity": "cosine"
    }]
})

# Define a query vector (like an embedding you'd get from an ML model)
query_vector = documents[0]["embedding"]  # Let's say we're searching similar to "cat"

# Perform a vector similarity search
result = db.command({
    "find": "embeddings",
    "filter": {
        "$vectorSearch": {
            "queryVector": query_vector,
            "path": "embedding",
            "k": 3,
            "similarity": "cosine"
        }
    }
})

print("Search results:")
for doc in result["cursor"]["firstBatch"]:
    print(doc)


OperationFailure: Error in specification { name: "embedding_vector_index", key: { embedding: "cosine" }, type: "vectorSearch", numDimensions: 5, similarity: "cosine" } :: caused by :: The field 'type' is not valid for an index specification. Specification: { name: "embedding_vector_index", key: { embedding: "cosine" }, type: "vectorSearch", numDimensions: 5, similarity: "cosine" }, full error: {'ok': 0.0, 'errmsg': 'Error in specification { name: "embedding_vector_index", key: { embedding: "cosine" }, type: "vectorSearch", numDimensions: 5, similarity: "cosine" } :: caused by :: The field \'type\' is not valid for an index specification. Specification: { name: "embedding_vector_index", key: { embedding: "cosine" }, type: "vectorSearch", numDimensions: 5, similarity: "cosine" }', 'code': 197, 'codeName': 'InvalidIndexSpecificationOption'}

In [7]:
from pymilvus import connections, utility, CollectionSchema, FieldSchema, DataType, Collection
import random

# 1. Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# 2. Define collection and schema
collection_name = "test"

if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=4)
]
schema = CollectionSchema(fields, description="Example schema")

collection = Collection(name=collection_name, schema=schema)

# 3. Insert data
ids = [1]
embeddings = [[random.random() for _ in range(4)]]
collection.insert([ids, embeddings])

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 457395598653128708, success count: 1, err count: 0)

In [8]:
ids = [2]
embeddings = [[random.random() for _ in range(4)]]
collection.insert([ids, embeddings])

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 457395603163316226, success count: 1, err count: 0)

In [10]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128}
}

collection.create_index(
    field_name="embedding",
    index_params=index_params
)

collection.load()

In [ ]:
from pymilvus import connections, utility, CollectionSchema, FieldSchema, DataType, Collection
import random

# 1. Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# 2. Define collection and schema
collection_name = "test"

if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=4)
]
schema = CollectionSchema(fields, description="Example schema")

collection = Collection(name=collection_name, schema=schema)

# 3. Insert data
ids = [1]
embeddings = [[random.random() for _ in range(4)]]
collection.insert([ids, embeddings])

#collection.load()

# 4. Search the inserted vector
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=embeddings,  # using the same vector we inserted
    anns_field="embedding",
    param=search_params,
    limit=1,
    output_fields=["id"]
)

print("Search results:")
for hits in results:
    for hit in hits:
        print(f"ID: {hit.entity.get('id')}, distance: {hit.distance}")


RPC error: [load_collection], <MilvusException: (code=700, message=index not found[collection=example_collection])>, <Time:{'RPC start': '2025-04-16 19:38:39.108985', 'RPC error': '2025-04-16 19:38:39.118726'}>


MilvusException: <MilvusException: (code=700, message=index not found[collection=example_collection])>

In [5]:
collection.load()
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=embeddings,  # using the same vector we inserted
    anns_field="embedding",
    param=search_params,
    limit=1,
    output_fields=["id"]
)

print("Search results:")
for hits in results:
    for hit in hits:
        print(f"ID: {hit.entity.get('id')}, distance: {hit.distance}")

RPC error: [load_collection], <MilvusException: (code=700, message=index not found[collection=example_collection])>, <Time:{'RPC start': '2025-04-16 19:46:47.700760', 'RPC error': '2025-04-16 19:46:47.712674'}>


MilvusException: <MilvusException: (code=700, message=index not found[collection=example_collection])>

In [6]:
# 5. Delete the entry
expr = "id in [1]"
collection.delete(expr)

# 6. Query to confirm deletion
collection.load()
results_after_delete = collection.query(expr=expr, output_fields=["id"])
print("Query after deletion:", results_after_delete)

RPC error: [load_collection], <MilvusException: (code=700, message=index not found[collection=example_collection])>, <Time:{'RPC start': '2025-04-16 19:47:42.770273', 'RPC error': '2025-04-16 19:47:42.775636'}>


MilvusException: <MilvusException: (code=700, message=index not found[collection=example_collection])>